In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held: 
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old, 
And see thy blood warm when thou feel'st it cold.""".split()

In [117]:
test_sentence = "當地警察已在28日下午開始監控人潮，但遊客數量似乎不如預期的多，因為當局目前認定還沒有關閉閘門的必要。威尼斯絡繹不絕的觀光客，已經成為當地人的惡夢。當地巷弄窄小，在旅遊旺季街道幾乎難以行走。"
test_sentence = str(test_sentence)

In [118]:
trigrams = [ ([test_sentence[i], test_sentence[i+1]], test_sentence[i+2])
            for i in range(len(test_sentence) - 2) ]

In [119]:
print(trigrams[:6])

[(['當', '地'], '警'), (['地', '警'], '察'), (['警', '察'], '已'), (['察', '已'], '在'), (['已', '在'], '2'), (['在', '2'], '8')]


In [120]:
vocab = set(test_sentence)
word_to_ix = {word : i for i, word in enumerate(vocab)}
ix_to_word = {i: word for i, word in enumerate(vocab)}
print(len(vocab))#每個詞向量長度 = 97

74


In [121]:
CONTEXT_SIZE = 2 
EMBEDDING_DIM = 10 

In [122]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embdeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embdeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs, self.embeddings

In [123]:
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)

In [124]:
losses = []
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [131]:
for epoch in range(2000):
    total_loss = torch.Tensor([0])# total_loss = 0
    
    for context, target in trigrams:
        # step 1:
        # 將context(ex : ['When','forty'])轉換成index(ex : [68, 15])
        # 再轉成pytorch的variable
        context_idxs = [word_to_ix[w] for w in context]
        context_var = autograd.Variable(torch.LongTensor(context_idxs))
        
        # step 2:
        # 清空gradient，防止上一次的累計
        model.zero_grad()
        
        # step 3:
        # 丟variable變數進去跑forward
        log_probs, embedd = model(context_var)
        
        # step 4:
        # 計算loss(把target variable丟進去)
        loss = loss_function(log_probs, autograd.Variable(torch.LongTensor([word_to_ix[target]])))
        
        # step 5:
        # 跑backward，更新gradient
        loss.backward()
        optimizer.step()
        
        total_loss += loss.data
    losses.append(total_loss)
    if epoch % 100 == 0:
        print(total_loss[0])

tensor(7.0881)
tensor(6.8002)
tensor(6.5768)
tensor(6.3990)
tensor(6.2546)
tensor(6.1353)
tensor(6.0352)
tensor(5.9502)
tensor(5.8771)
tensor(5.8137)
tensor(5.7582)
tensor(5.7094)
tensor(5.6660)
tensor(5.6271)
tensor(5.5922)
tensor(5.5607)
tensor(5.5321)
tensor(5.5060)
tensor(5.4821)
tensor(5.4602)


In [151]:
word, target_word = trigrams[1]
print('input context = ', word)
word = autograd.Variable(torch.LongTensor([word_to_ix[i] for i in word]))
out, outEmbedd = model(word)
# torch.max : Returns the maximum value of each row of the input tensor 
# in the given dimension dim(第二個參數). 
# The second return value is the index location of each maximum value found.
_, predict_label = torch.max(out,1)
print(predict_label)
predict_word = ix_to_word[predict_label.data[0][0].item()]
print('real word is {}, predict word is {}'.format(target_word, predict_word))

input context =  ['地', '警']
tensor([ 59])
real word is 察, predict word is 察


C:\Users\miplab01\Anaconda3\envs\jiebaTest\lib\site-packages\ipykernel_launcher.py:10: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  # Remove the CWD from sys.path while we load stuff.


In [59]:
for i in range(0, len(vocab)):
    lookup_tensor = torch.LongTensor([i])
    print(ix_to_word[i])
    print(embedd(autograd.Variable(lookup_tensor)))

asked,
tensor([[-1.1568, -0.9825,  0.3531,  0.8147, -2.9700,  0.0928,  1.0171,
          0.6419,  1.0057, -0.7808]])
own
tensor([[ 0.6147,  0.1798, -2.4165, -3.5395,  0.6888, -1.2381, -0.4775,
         -0.6608,  0.0082, -0.4247]])
by
tensor([[ 1.6343,  1.3534, -1.5896, -0.4293, -0.3689,  1.0900,  0.8851,
         -0.2824,  1.9333,  0.0354]])
Where
tensor([[-1.3096,  0.1470, -0.1365,  0.8540,  1.0137,  0.1830,  0.1384,
          0.1679, -1.3315,  1.1201]])
How
tensor([[ 0.0886, -1.0286, -0.4590, -1.5353, -0.3357,  0.4436,  0.8429,
          0.6780,  0.3049, -0.1948]])
totter'd
tensor([[ 1.3638,  0.5667,  0.9565,  1.4773, -0.7453,  1.6480, -0.4604,
          0.3771, -1.0529,  1.7641]])
Thy
tensor([[ 0.0542,  1.5121,  0.0833,  0.1231, -1.2143, -0.8155, -0.0919,
         -1.7810, -0.0789, -1.0564]])
thy
tensor([[ 1.5832,  2.1953,  0.9612,  0.4272, -1.9099,  0.2299, -0.7458,
         -1.6041, -0.2862, -0.6104]])
made
tensor([[ 0.2354, -1.6590, -1.1956,  0.0889,  0.0549,  1.7187, -0.8777,
  